

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/akshayrb22/playing-with-data/blob/master/supervised_learning/KNN/KNN.ipynb)

# K-Nearest Neighbors

 K-Nearest Neighbors, is a simple machine learning algorithm used for classification and regression. It works by finding the K closest data points to a new one and predicting its value based on what those neighbors are, kind of like asking nearby people for their opinion and going with the majority.

The algorithm calculates distance between points, finds the nearest neighbors, and then either takes a vote (for classification) or an average (for regression). It doesn’t require training, which makes it easy to use, but it can be slow with large datasets and sensitive to outliers or unscaled features.

K-NN is used in real-world applications like credit scoring.

In [4]:
#Importing necessary libraries
import pandas as pd
import numpy as np
import math
import operator

# Importing the Dataset

In [5]:
import pandas as pd

df = pd.read_csv('/content/LasVegasTripAdvisorReviews-Dataset.csv', sep=';')


# Train-Test Split

In [7]:
np.random.seed(1)
msk = np.random.rand(
    len(df)) < 0.7  #An array containing True(with probability 0.7) and False
train = df[msk]  #Rows having array value true
test = df[~msk]  #Rows having array value False
print('Number of observations in the training data: ', len(train))
print('Number of observations in the test data: ', len(test))

Number of observations in the training data:  347
Number of observations in the test data:  157


In [8]:
train.head()

,User country,Nr. reviews,Nr. hotel reviews,Helpful votes,Score,Period of stay,Traveler type,Pool,Gym,Tennis court,Spa,Casino,Free internet,Hotel name,Hotel stars,Nr. rooms,User continent,Member years,Review month,Review weekday
0,USA,11,4,13,5,Dec-Feb,Friends,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3,3773,North America,9,January,Thursday
2,USA,36,9,25,5,Mar-May,Families,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3,3773,North America,2,February,Saturday
3,UK,14,7,14,4,Mar-May,Friends,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3,3773,Europe,6,February,Friday
4,Canada,5,5,2,4,Mar-May,Solo,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3,3773,North America,7,March,Tuesday
5,Canada,31,8,27,3,Mar-May,Couples,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3,3773,North America,2,March,Tuesday


In [9]:
test.head()

,User country,Nr. reviews,Nr. hotel reviews,Helpful votes,Score,Period of stay,Traveler type,Pool,Gym,Tennis court,Spa,Casino,Free internet,Hotel name,Hotel stars,Nr. rooms,User continent,Member years,Review month,Review weekday
1,USA,119,21,75,3,Dec-Feb,Business,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3,3773,North America,3,January,Friday
13,USA,22,5,13,3,Jun-Aug,Friends,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3,3773,North America,1,July,Thursday
20,UK,10,5,2,4,Sep-Nov,Couples,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3,3773,Europe,7,November,Saturday
21,New Zeland,4,3,3,1,Sep-Nov,Couples,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3,3773,Oceania,3,November,Monday
24,Ireland,29,11,15,4,Dec-Feb,Couples,YES,YES,NO,YES,YES,YES,Excalibur Hotel & Casino,3,3981,Europe,3,January,Monday


# Euclidean Distance

It is used to measure the distance between samples p and q in an n-dimensional feature space:

<img src="https://drive.google.com/uc?export=view&id=13p3YOEqHybBrN8P5wpsPwBC6-7v3N98f">

For example, picture it as a "straight, connecting" line in a 2D feature space:

<img src="https://drive.google.com/uc?export=view&id=1Qc0nBRJTWBrJeKZvvj8dVk7YZJrvMpn8">

In [ ]:
def euclideanDistance(instance1, instance2, length):
    distance = 0
    for x in range(length):
        distance += pow((instance1[x] - instance2[x]), 2)
    return math.sqrt(distance)

# Calculating the Nearest Neighbors

Let us consider that our data is scattered as shown below. Let the red circles denote Malignant and the green circles denote Benign:

<img src="https://drive.google.com/uc?export=view&id=1oEbIjxE0Ysdo5fEeNU5CGo4ruP544Ywz">

In this example we are calculating k=3 nearest neighbors so the curve of fit will look like:

<img src="https://drive.google.com/uc?export=view&id=1amCw22CqF0ideK9VWNIWQAASpZOWGPwi">

In [ ]:
def getNeighbors(trainingSet, testInstance, k):
    distances = []  #List to store all the distance values
    length = len(testInstance) - 1
    for x in range(len(trainingSet)):
        dist = euclideanDistance(testInstance, trainingSet[x], length)  #Calculating the Euclidean Distance
        distances.append((trainingSet[x],dist))  #Appending the distance values to the 'distance' list
    distances.sort(key=operator.itemgetter(1))  #Sorting based on the disance value
    neighbors = []  #List to store all the neighbors
    for x in range(k):
        neighbors.append(distances[x][0])  #Number of neighbors is dependent on the value of k
    return neighbors

In [ ]:
def getResponse(neighbors):
    classVotes = {}  #Dictionary to store labels with their counts
    for x in range(len(neighbors)):
        response = neighbors[x][-1]  #Label value of the neighbors
        if response in classVotes:
            classVotes[
                response] += 1  #if label value is already present increment it by 1
        else:
            classVotes[
                response] = 1  #If the label value is not yet present add it to the dictionary
    sortedVotes = sorted(
        classVotes.items(), key=operator.itemgetter(1), reverse=True
    )  #Sort the dictinary based on the count value in descending order
    return sortedVotes[0][
        0]  #Return the label with highest number of occurences

# Calculating the accuracy

Accuracy is one metric for evaluating machine learning models. Informally, accuracy is the fraction of predictions our model got right. Formally, accuracy has the following definition:

<img src="https://drive.google.com/uc?export=view&id=1gk-T6kanOB6mM_bs54jgyMs0Bu93V4sh">

In [ ]:
def getAccuracy(testSet, predictions):
    correct = 0  #Variable to store the correct predictions
    for x in range(len(testSet)):
        if testSet[x][-1] is predictions[x]:  #Checking whether the predicted value is same as label value
            correct += 1  #Incremented when both values are same
    return (correct / float(len(testSet))) * 100.0  #Accuracy = No. of Correct pred / Total number of pred

# Implementation on a smaller dataset

In [ ]:
trainSet = [[5, 1, 1, 1, 2, 1, 3, 2, 1, 2],
            [10, 10, 10, 10, 5, 10, 10, 10, 7, 4]]
testInstance = [4, 8, 6, 4, 3, 4, 10, 6, 1, 2]
k = 1
neighbors = getNeighbors(trainSet, testInstance, k)
print(neighbors)

[[5, 1, 1, 1, 2, 1, 3, 2, 1, 2]]


In [ ]:
neighbors = [[5, 1, 1, 1, 2, 1, 3, 2, 1, 2], [3, 1, 1, 1, 2, 1, 2, 3, 1, 2],
             [10, 10, 10, 10, 5, 10, 10, 10, 7, 4]]
response = getResponse(neighbors)
print(response)

2


In [ ]:
testSet = [[5, 1, 1, 1, 2, 1, 3, 2, 1, 2], [3, 1, 1, 1, 2, 1, 2, 3, 1, 2],
           [10, 10, 10, 10, 5, 10, 10, 10, 7, 4]]
predictions = [2, 2, 2]
accuracy = getAccuracy(testSet, predictions)
print(accuracy)

66.66666666666666


# Implementation on the imported dataset

In [ ]:
predictions = []  #List to store the predicted values
k = 3  # 3-Nearest Neighbors
trainingSet = train.values.tolist()  #List containing training data
testSet = test.values.tolist()  #List containing test data
for x in range(len(testSet)):
    neighbors = getNeighbors(trainingSet, testSet[x], k)
    result = getResponse(neighbors)
    predictions.append(result)  # Storing the predicted values
    print('> predicted=' + repr(result) + ', actual=' + repr(testSet[x][-1]))

> predicted=1.0, actual=1.0
> predicted=2.0, actual=1.0
> predicted=2.0, actual=2.0
> predicted=2.0, actual=2.0
> predicted=1.0, actual=1.0
> predicted=1.0, actual=1.0
> predicted=1.0, actual=1.0
> predicted=1.0, actual=1.0
> predicted=2.0, actual=2.0
> predicted=2.0, actual=1.0
> predicted=1.0, actual=1.0
> predicted=2.0, actual=1.0
> predicted=1.0, actual=1.0
> predicted=2.0, actual=2.0
> predicted=2.0, actual=2.0
> predicted=1.0, actual=1.0
> predicted=1.0, actual=1.0
> predicted=2.0, actual=2.0
> predicted=1.0, actual=1.0
> predicted=2.0, actual=2.0
> predicted=2.0, actual=2.0
> predicted=1.0, actual=1.0
> predicted=1.0, actual=1.0
> predicted=1.0, actual=1.0
> predicted=2.0, actual=1.0
> predicted=2.0, actual=2.0
> predicted=2.0, actual=2.0
> predicted=2.0, actual=2.0
> predicted=2.0, actual=2.0
> predicted=2.0, actual=2.0
> predicted=2.0, actual=2.0
> predicted=2.0, actual=2.0
> predicted=2.0, actual=2.0
> predicted=1.0, actual=1.0
> predicted=1.0, actual=1.0
> predicted=2.0, act

# Confusion Matrix

Confusion Matrix is a performance measurement for machine learning classification problem where output can be two or more classes. It is a table with 4 different combinations of predicted and actual values.

<img src="https://drive.google.com/uc?export=view&id=1hYthqmYW82QHRFnIpiGQDLt4r44aurv7">

In [ ]:
print("Confusion Matrix")
y_test = []
for i in testSet:
    y_test.append(i[30])
from sklearn.metrics import confusion_matrix, accuracy_score
res = confusion_matrix(y_test, predictions)
print(res)

Confusion Matrix
[[ 52   5]
 [  6 111]]


# Accuracy

In [ ]:
accuracy = accuracy_score(y_test, predictions) * 100
print('Accuracy: ' + repr(accuracy) + '%')

Accuracy: 93.67816091954023%
